## 计算一些风格因子

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt

/mnt/datadisk2/aglv/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
cd /mnt/datadisk2/aglv/aglv/lab_aglv/forintern/

/mnt/datadisk2/aglv/aglv/lab_aglv/forintern


In [69]:
path = '/mnt/datadisk2/aglv/aglv/aglv_factor/style_factor/'

In [4]:
from DataDaily import DataDaily
datadaily = DataDaily()

load ./data_daily.h5...


In [44]:
close = datadaily.adjclose.loc[:20231231]
vol = datadaily.volume.loc[:20231231]
ret = (close - close.shift(1)) / close.shift(1)
momentum20 = (close - close.shift(20)) / close.shift(20)

In [20]:
momentum20.to_csv(path + 'momentum20.csv')

In [7]:
mkt_cap = np.log(datadaily.mkt_cap)

In [11]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [27]:
x = np.random.rand(100, 2)
y = 2 * x[:, 0] + 3*x[:, 1] + 5 + np.random.rand(100) * 1
x = sm.add_constant(x)
model = sm.OLS(y, x)
res = model.fit()


In [28]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     605.2
Date:                Wed, 13 Nov 2024   Prob (F-statistic):           1.63e-55
Time:                        13:49:45   Log-Likelihood:                -17.496
No. Observations:                 100   AIC:                             40.99
Df Residuals:                      97   BIC:                             48.81
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.4877      0.079     69.614      0.000       5.331       5.644
x1             2.1113      0.100     21.192      0.000       1.914       2.309
x2             2.9139      0.100     29.188      0.000       2.716       3.112
==============================================================================
Omnibus:                       31.060   Durbin-Watson:                   2.129
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                6.019
Skew:                           0.110   Prob(JB):                       0.0493
Kurtosis:                       1.819   Cond. No.                         5.40
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
res.resid

array([ 0.46263661, -0.12537014, -0.32473174,  0.26415255,  0.18089807,
       -0.05901024,  0.01553827,  0.22898024, -0.32110101,  0.48185187,
        0.532966  ,  0.13053957, -0.308182  ,  0.18571963, -0.43460075,
       -0.46112659, -0.17188909, -0.11959881,  0.29974598,  0.26613656,
        0.3074806 , -0.11114956, -0.11781428, -0.10088823,  0.45197597,
       -0.47169264,  0.30173535,  0.05285236, -0.12683874,  0.24735346,
        0.06818849,  0.31498866,  0.44042777, -0.02340835, -0.19595398,
        0.17453123,  0.39970865, -0.34928613,  0.05297975,  0.1692079 ,
        0.25731585, -0.39381556,  0.28346271,  0.41272583, -0.25785462,
       -0.308373  ,  0.53305695, -0.35606363,  0.27385659,  0.03765342,
       -0.23225029,  0.36237413, -0.11667707, -0.05260524, -0.14953986,
        0.06442646, -0.37965264, -0.18840828,  0.45187428, -0.24266623,
        0.13822614, -0.17435626,  0.23635814, -0.40404216,  0.47980055,
       -0.07318867, -0.23169072, -0.37130014, -0.27926577,  0.03

InstrumentID  date    
BJ430017      20171009   NaN
              20171010   NaN
              20171011   NaN
              20171012   NaN
              20171013   NaN
                          ..
SZ301633      20231225   NaN
              20231226   NaN
              20231227   NaN
              20231228   NaN
              20231229   NaN
Length: 8549812, dtype: float64

In [54]:
ret_stack = ret.loc[20210101:20231231].unstack()
momentum20_stack = momentum20.loc[20210101:20231231].unstack()

In [55]:
ret_mon20 = pd.concat([ret_stack, momentum20_stack], axis=1)
ret_mon20

0   1
InstrumentID date            
BJ430017     20210104 NaN NaN
             20210105 NaN NaN
             20210106 NaN NaN
             20210107 NaN NaN
             20210108 NaN NaN
...                    ..  ..
SZ301633     20231225 NaN NaN
             20231226 NaN NaN
             20231227 NaN NaN
             20231228 NaN NaN
             20231229 NaN NaN

[4097372 rows x 2 columns]

In [58]:
def f(x, y):
    if x.isna().sum() + y.isna().sum() == 0:
        x = sm.add_constant(x)
        model = sm.OLS(y, x)
        res = model.fit()
        return res.resid[-1]
    else:
        return np.nan

In [60]:
# res = (ret_mon20
#        .groupby(level='InstrumentID')
#        .progress_apply(lambda x: x.iloc[:, 0].rolling(20).apply(lambda y:f(x.iloc[:, 1].loc[y.index], y)))
#     )

In [64]:
import multiprocessing
import statsmodels.api as sm
shift_day = 0
r_win = 20
n_processing = 64

def _handle_stockly(data_stock: pd.DataFrame, r_win=r_win):
    def f(x, y):
        if x.isna().sum() + y.isna().sum() == 0:
            x = sm.add_constant(x)
            model = sm.OLS(y, x)
            res = model.fit()
            return res.resid[-1]
        else:
            return np.nan

    return data_stock.iloc[:, 0].rolling(r_win).apply(lambda x:f(x, data_stock.iloc[:, 1].loc[x.index]))

def process_group(group, r_win=15):
    instrument_id, group_data = group
    return (instrument_id, _handle_stockly(group_data, r_win))

def multiprocessing_factor(data: pd.DataFrame, n_processes=n_processing):
    grouped = list(data.groupby(level='InstrumentID'))
    with multiprocessing.Pool(processes=n_processes) as pool:
        results = list(tqdm(
            pool.imap(process_group, grouped), total=len(grouped)
        ))

    factor_list = []
    for instrument_id, factor in results:
        factor_df = pd.DataFrame(
            factor.values,
            index=factor.index.get_level_values('date'), 
            columns=[factor.index[0][0]]
        )
        factor_list.append(factor_df)

    factor = pd.concat(factor_list, axis=1)

    return factor

In [65]:
FA = multiprocessing_factor(ret_mon20)

In [68]:
FA.isna().sum(axis=1)

date
20210104    5636
20210105    5636
20210106    5636
20210107    5636
20210108    5636
            ... 
20231225     370
20231226     372
20231227     371
20231228     373
20231229     370
Length: 727, dtype: int64

In [70]:
FA.to_csv(path + 'ret_mon20_olsresid.csv')

In [ ]:
def ols_resid(fa1:pd.DataFrame, fa2:pd.DataFrame):
    import statsmodels.api as sm

    u = set(fa1.columns).intersection(set(fa2.columns))
    fa1 = fa1.loc[:, u].unstack().stack(level='InstrumentID').stack(level='date')
    fa2 = fa2.loc[:, u].unstack().stack(level='InstrumentID').stack(level='date')

    def _ols_resid(x, y):
        if x.isna().sum() + y.isna().sum() == 0:
            x = sm.add_constant(x)
            model = sm.OLS(y, x)
            res = model.fit()
            return res.resid[-1]
        else:
            return np.nan